In [141]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout



In [52]:
!pip install matplotlib


In [143]:
train_with_label_path = 'train/with_label'
train_no_label_path = 'train/no_label'
test_path = 'test'

img_height, img_width = 150, 150
batch_size = 32
num_classes = 2  #deux types : Clean and Dirty

In [144]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
#data labélé
train_with_label_gen = train_datagen.flow_from_directory(
    train_with_label_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

#data non labélé
unlabeled_datagen = ImageDataGenerator(rescale=1.0/255.0)
unlabeled_gen = unlabeled_datagen.flow_from_directory(
    train_no_label_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode=None,  # No labels
    shuffle=False
)
train_data = train_with_label_gen


Found 40 images belonging to 2 classes.
Found 1 images belonging to 1 classes.


In [146]:
#création de modéle CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

#compilation de modéle 
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#train do model 
model.fit(train_data, epochs=20)

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 667ms/step - accuracy: 0.5000 - loss: 0.8415
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 487ms/step - accuracy: 0.5833 - loss: 1.3752
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.5167 - loss: 0.6982 
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.5104 - loss: 0.7165 
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 494ms/step - accuracy: 0.7000 - loss: 0.6797
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.5437 - loss: 0.6853 
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 513ms/step - accuracy: 0.4667 - loss: 0.6847
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.6562 - loss: 0.6965 
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 465ms/step - accuracy: 0.5750 - loss: 0.6767
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.6729 - loss: 0.6728 
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - accuracy: 0.6625 - loss: 0.6716 
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 455ms/step - accuracy: 0.7917 - lo

In [152]:
model.save('classification_poubelle.h5')

In [150]:
#evaluation de modéle avec les imaes test 
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)

# Make predictions on test data using the test generator
predictions = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size)

# Convert predictions to class labels (0 or 1)
predicted_classes = (predictions > 0.5).astype(int)

# Get the filenames of the test images
filenames = test_generator.filenames

# Print predicted classes for test images with filenames
for filename, predicted_class in zip(filenames, predicted_classes.flatten()):
    print(f"Image: {filename} - Predicted class: {'dirty' if predicted_class == 1 else 'clean'}")



Found 100 images belonging to 1 classes.
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 246ms/step
Image: test\0003717662-666x388.jpg - Predicted class: clean
Image: test\00546_01.jpg - Predicted class: dirty
Image: test\00566_02.jpg - Predicted class: dirty
Image: test\00569_03.jpg - Predicted class: clean
Image: test\00594_00.jpg - Predicted class: dirty
Image: test\00798_02.jpg - Predicted class: clean
Image: test\00829_07.jpg - Predicted class: dirty
Image: test\01300_00.jpg - Predicted class: clean
Image: test\01670_07.jpg - Predicted class: clean
Image: test\01787_01.jpg - Predicted class: dirty
Image: test\01906_01.jpg - Predicted class: clean
Image: test\02136_04.jpg - Predicted class: dirty
Image: test\02248_00.jpg - Predicted class: dirty
Image: test\02587_03.jpg - Predicted class: clean
Image: test\02752_02.jpg - Predicted class: clean
Image: test\02887_03.jpg - Predicted class: dirty
Image: test\03060_07.jpg - Predicted class: clean
Image: test\03213_03.jpg - Predicted class: dirty
Image: tes

In [153]:
#deploiment de modéle pour l'application mobile
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open('classification_poubelle.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\USER\AppData\Local\Temp\tmppa_r4oj4\assets


INFO:tensorflow:Assets written to: C:\Users\USER\AppData\Local\Temp\tmppa_r4oj4\assets


Saved artifact at 'C:\Users\USER\AppData\Local\Temp\tmppa_r4oj4'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name='keras_tensor_1455')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  2410101393888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410101393712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410102101264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410102101088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410102103200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410102102848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410102126016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410102125840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410102127952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410102127600: TensorSpec(shape=(), dtype=tf.resource, name=None)
